# Imports

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

from tensorflow.keras import backend as K
import gc

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import torch
from diffusers import StableDiffusionImg2ImgPipeline

from tqdm import tqdm

# Parameters

In [2]:
# Number of images to robustify by stable diffusion (SD)
n_rows = 5000

In [3]:
# NN Parameters
epochs = 25
batch_size = 16

In [4]:
# Strength of vision model changing of the image
strengths = [0.01, 0.03, 0.06]

# Loading data

In [5]:
# We train on equal size, same images, just the base setup is regular images whilst the robustified NN uses the robustified images
train, test = train_test_split(pd.read_csv("houses_preprocessed.csv"), train_size=n_rows, shuffle=True, random_state=42)

# Image Robustification

In [6]:
# Load the Stable Diffusion img2img pipeline
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda") # Use cuda
pipe.set_progress_bar_config(disable=True) # Disable progress bar per image

# Create output directories
for strength in strengths:
    os.makedirs(os.path.join("Images Robustified", f"{model_id}", f"Strength-{strength}"), exist_ok=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
# Robustifying Images
for strength in strengths:
    for index, row in tqdm(train.iterrows(), total=n_rows, desc=f"Robustifying Images with Strength {strength}"):   
        # Load the image to be robustified
        input_image = Image.open(row["image"]).convert("RGB")
        
        # Define the prompt for img2img
        prompt = "Enhance core features and remove irrelevant elements to strengthen image clarity and focus"
        
        # Run the pipeline
        result = pipe(prompt=prompt, image=input_image, strength=strength).images[0]
    
        # LANCZOS resizing
        original_width, original_height = input_image.size
        result_resized = result.resize((original_width, original_height), Image.LANCZOS)
        
        # Save the result
        output_path = os.path.join("Images Robustified", f"{model_id}", "Strength-" + str(strength), f"{index}_{row['price']}.jpg")
        result_resized.save(output_path)

Robustifying Images with Strength 0.06: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [2:44:26<00:00,  1.97s/it]


# Experimental set up

## Train and Test the models on the same data partioning

In [8]:
# Train data
X_train_tab = train[['n_citi', 'bed', 'bath', 'sqft']].values 
X_train_img = train['image'] # pd
y_train = train['price']

# Test data ith compression cols
X_test_tab = test[['n_citi', 'bed', 'bath', 'sqft']].values 
X_test_img = test['image'] # pd
y_test = test['price']

# Print shapes
print("Training Data Shapes:")
print(f"Tabular features: {X_train_tab.shape}")
print(f"Image features: {X_train_img.shape}")
print(f"Target prices: {y_train.shape}")
print("\nTest Data Shapes:")
print(f"Tabular features: {X_test_tab.shape}")
print(f"Image features: {X_test_img.shape}")
print(f"Target prices: {y_test.shape}")

Training Data Shapes:
Tabular features: (5000, 4)
Image features: (5000,)
Target prices: (5000,)

Test Data Shapes:
Tabular features: (10297, 4)
Image features: (10297,)
Target prices: (10297,)


# Neural Networks and Models

## Base NN and Resnet

In [9]:
def base_nn(image_shape=(311, 415, 3)):
    # Image processing branch
    img_input = Input(shape=image_shape, name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(img_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction

    # Define the model
    nn_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    nn_model.compile(optimizer='adam',
                  loss='mae',
                  metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # nn_model.summary()

    return nn_model

In [10]:
def resnet_nn(image_shape=(311, 415, 3)):
    # Image processing branch with pre-trained ResNet50
    res_net = ResNet50(weights='imagenet', include_top=False, input_shape=image_shape)
   
    # Unfreeze only the last 10 layers of resnet (fine-tuning) 
    res_net.trainable = False 
    for layer in res_net.layers[-10:]:
        layer.trainable = True

    # Image processing branch
    img_input = Input(shape=image_shape, name='image_input')
    x = res_net(img_input)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction
    
    # Define the model
    res_net_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    res_net_model.compile(optimizer='adam', 
                          loss='mae',
                          metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # res_net_model.summary()

    return res_net_model

In [11]:
'''
I did not write this code, the code is from: https://www.tensorflow.org/tutorials/load_data/images
It helps us train the NN more dynamically, it loads images on the go, such that not all RAM is used up.
It does try to maximise RAM usage this is basically what the tf.data.AUTOTUNE does.
'''

# Loads an image and normalizes it from [0,1]
def process_example(image_path, label):
    # Load raw bytes and convert to RGB
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize image to [0, 1] and convert to float32
    image = tf.image.convert_image_dtype(image, tf.float32)

    return image, label


# Creates on the fly data sets to train/test the model, we need this to not exceed memory
def create_dataset(image_paths, labels, shuffle=True):
    # Convert to tensors
    image_paths = tf.convert_to_tensor(image_paths)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    # Build dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda img, lbl: process_example(img, lbl), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset


def train_and_evaluate_nn(nn, 
                          X_train_img_paths, y_train,
                          X_test_img_paths, y_test,
                          verbose=1):

    # Dynamic dataset loading
    train_robustified = create_dataset(X_train_img_paths, y_train, shuffle=True) # Shuffle to break ordering
    test_robustified = create_dataset(X_test_img_paths, y_test, shuffle=False) # No shuffle, we arent learning, just predicting

    # Train and Test
    history = nn.fit(train_robustified, epochs=epochs, verbose=verbose)
    test_loss, test_mae, r2 = nn.evaluate(test_robustified, verbose=0)

    return history, test_loss, test_mae, r2

## Logistic regression

In [12]:
def train_and_evaluate_lin_model(model, X_train_tab, y_train, X_test_tab, y_test):
    # Train the model
    model.fit(X_train_tab, y_train)
    
    # Evaluate the model
    y_test_pred = model.predict(X_test_tab)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    
    return mae_test, r2

### Train and Evaluate

In [13]:
# Create NNs with tabular features = 4 (n_citi, bed, bath, sqft)
nn_base = base_nn()
nn_resnet = resnet_nn()
lin = LinearRegression()

In [14]:
# NN
print("Training Base NN")
nn_base_hist, _, nn_base_mae, nn_base_r2 = train_and_evaluate_nn(nn_base, X_train_img, y_train, X_test_img, y_test)
print(f"NN Base MAE: {nn_base_mae:.0f}\nNN Base R2: {nn_base_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Base NN
Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - R2Score: -2.0227 - loss: 536502.8750 - mae: 536502.8750
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - R2Score: -0.1614 - loss: 285796.9375 - mae: 285796.9375
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 374s 1s/step - R2Score: -0.1538 - loss: 287313.7500 - mae: 287313.7500
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - R2Score: -0.1531 - loss: 278373.6250 - mae: 278373.6250
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step - R2Score: -0.1457 - loss: 289209.5312 - mae: 289209.5312
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - R2Score: -0.1451 - loss: 283187.7812 - mae: 283187.7812
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - R2Score: -0.1102 - loss: 284971.5312 - mae: 284971.5312
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - R2Score: -0.1206 - loss: 280774.8438 - mae: 280774.8438
Epoch 9/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - R2Score: -0.1259 - loss: 285588.2500 - m

0

In [15]:
# Resnet
print("Training Resnet")
nn_resnet_hist, _, nn_resnet_mae, nn_resnet_r2 = train_and_evaluate_nn(nn_resnet, X_train_img, y_train, X_test_img, y_test)
print(f"Resnet MAE: {nn_resnet_mae:.0f}\nResnet R2: {nn_resnet_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Resnet
Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 807s 3s/step - R2Score: -3.6518 - loss: 709057.3125 - mae: 709057.3125
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 797s 3s/step - R2Score: -3.1413 - loss: 669024.5625 - mae: 669024.5625
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 797s 3s/step - R2Score: -1.9727 - loss: 522294.0312 - mae: 522294.0312
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 814s 3s/step - R2Score: -0.1935 - loss: 282823.2500 - mae: 282823.2500
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 806s 3s/step - R2Score: 0.0399 - loss: 256436.0312 - mae: 256436.0312
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 796s 3s/step - R2Score: 0.0807 - loss: 253381.1562 - mae: 253381.1562
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 797s 3s/step - R2Score: 0.0559 - loss: 259391.8750 - mae: 259391.8750
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 809s 3s/step - R2Score: 0.0660 - loss: 251798.1094 - mae: 251798.1094
Epoch 9/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 805s 3s/step - R2Score: 0.0948 - loss: 252083.5000 - mae: 25

0

In [16]:
# LR
print("Training LR")
lr_mae, lr_r2 = train_and_evaluate_lin_model(lin, X_train_tab, y_train, X_test_tab, y_test)
print(f"LR MAE: {lr_mae:.0f}\nLR R2: {lr_r2:.2f}")

Training LR
LR MAE: 223691
LR R2: 0.35


# Comparison Robustified Images

In [17]:
def load_robustified_data(strength):
    # Load image paths and prices
    image_path = f"Images Robustified/{model_id}/Strength-{strength}"
    X_train_robustified = []
    y_train_robustified = []
    
    for img_file in os.listdir(image_path):
        if img_file.endswith('.jpg'):
            # Load image
            image = image_path + "/" + img_file
            X_train_robustified.append(image)
            
            # Extract price from filename (format: "i-price.jpg")
            price_str = img_file.split('.')[0].split('_')[1]  # Gets the price part
            y_train_robustified.append(int(price_str))
    
    # Ensure loaded images match the expected number of robustified images
    assert len(X_train_robustified) == n_rows
    assert len(y_train_robustified) == n_rows

    return X_train_robustified, y_train_robustified

In [18]:
def create_comparison(llm, model_results):
    # DF Structure
    comparison_data = {
        'Model': [],
        'MAE': [],
        'R2': []
    }
    
    for model_name, mae, r2 in model_results:
        comparison_data['Model'].append(model_name)
        comparison_data['MAE'].append(round(mae))
        comparison_data['R2'].append(round(r2, 3))
    
    # Make into df
    comparison_df = pd.DataFrame(comparison_data).set_index("Model")
    
    # Display df
    print(f"Comparison of Models for {llm}")
    display(comparison_df)
    print()
    return comparison_df

In [19]:
import logging

# Supress retracing warning
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # '0' = all messages, '3' = fatal only
tf.get_logger().setLevel(logging.ERROR)

In [20]:
# Initialize a list to store results
model_results = []

for strength in strengths:
    print(f"Training models for Strength: {strength}")
    
    # Load robustified images
    X_train_robustified, y_train_robustified = load_robustified_data(strength)
    
    # Create, Train and Evaluate the models
    nn_base_robustified = base_nn()
    nn_resnet_robustified = resnet_nn()
    
    print("Training Base NN")
    _, _, nn_base_robustified_mae, nn_base_robustified_r2 = train_and_evaluate_nn(nn_base_robustified, 
                                                                                  X_train_robustified, y_train_robustified,
                                                                                  X_test_img, y_test)
    print("Training Resnet NN")
    _, _, nn_resnet_robustified_mae, nn_resnet_robustified_r2 = train_and_evaluate_nn(nn_resnet_robustified, 
                                                                                      X_train_robustified, y_train_robustified, 
                                                                                      X_test_img, y_test)
    
    # Store results for both models
    model_results.append((f"NN Base Robustified SD-v1-5 Strength {strength}" , nn_base_robustified_mae, nn_base_robustified_r2))
    model_results.append((f"NN Resnet Robustified SD-v1-5 Strength {strength}", nn_resnet_robustified_mae, nn_resnet_robustified_r2))
    
    # Try to clear NN from memory
    K.clear_session()
    gc.collect()

Training models for Strength: 0.01
Training Base NN
Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - R2Score: -2.1810 - loss: 542455.0000 - mae: 542455.0000
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - R2Score: -0.1411 - loss: 282937.9688 - mae: 282937.9688
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 381s 1s/step - R2Score: -0.1656 - loss: 286460.8438 - mae: 286460.8438
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 375s 1s/step - R2Score: -0.1522 - loss: 280033.2188 - mae: 280033.2188
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 374s 1s/step - R2Score: -0.1452 - loss: 278261.5312 - mae: 278261.5312
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step - R2Score: -0.1480 - loss: 285512.5938 - mae: 285512.5938
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 378s 1s/step - R2Score: -0.1345 - loss: 282140.0000 - mae: 282140.0000
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - R2Score: -0.1325 - loss: 281779.4375 - mae: 281779.4375
Epoch 9/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - R2Sco

In [21]:
# Store default results
model_results.append(('NN Base', nn_base_mae, nn_base_r2))
model_results.append(('NN Resnet', nn_resnet_mae, nn_resnet_r2))

# Create a comparison of model results
comparison_table = create_comparison(model_id, model_results)

Comparison of Models for runwayml/stable-diffusion-v1-5


,MAE,R2
Model,,
NN Base Robustified SD-v1-5 Strength 0.01,268078,-0.026
NN Resnet Robustified SD-v1-5 Strength 0.01,253927,0.090
NN Base Robustified SD-v1-5 Strength 0.03,268075,-0.019
NN Resnet Robustified SD-v1-5 Strength 0.03,361489,-0.265
NN Base Robustified SD-v1-5 Strength 0.06,271860,-0.087
NN Resnet Robustified SD-v1-5 Strength 0.06,271547,0.091
NN Base,267975,-0.029
NN Resnet,277564,-0.147
